In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(42)

In [2]:
import pandas as pd

data_a = pd.read_csv('training_set_features.csv')
labels_a = pd.read_csv('training_set_labels.csv')
data_b = pd.read_csv('test_set_features.csv')

X_train_a = data_a.drop(columns=['respondent_id'])
y_train_a = labels_a.drop(columns=['respondent_id'])
X_test_b = data_b.drop(columns=['respondent_id'])

test_id_b = data_b['respondent_id']


In [3]:
missing_values_train_a = X_train_a.isnull().sum()
print(f"Missing values in training set:\n{missing_values_train_a[missing_values_train_a > 0]}")

missing_values_test_b = X_test_b.isnull().sum()
print(f"Missing values in test set:\n{missing_values_test_b[missing_values_test_b > 0]}")

missing_train_a = X_train_a.isnull().sum()
missing_test_b = X_test_b.isnull().sum()

print("Missing values in training set:\n", missing_train_a[missing_train_a > 0])
print("Missing values in test set:\n", missing_test_b[missing_test_b > 0])


Missing values in training set:
xyz_concern                       92
xyz_knowledge                    116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_xyz                 2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_xyz_vacc_effective       391
opinion_xyz_risk                 388
opinion_xyz_sick_from_vacc       395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
education                       1407
income_poverty                  4423
marital_status                  1408
rent_or_own                     2042
employment_status               1463
househ

In [4]:
threshold_a = 0.45
cols_to_drop_a = missing_train_a[missing_train_a > threshold_a * len(X_train_a)].index
X_train_a.drop(columns=cols_to_drop_a, inplace=True)
X_test_b.drop(columns=cols_to_drop_a, inplace=True)

from scipy.stats import skew

numeric_features_train_a = X_train_a.select_dtypes(include=['int64', 'float64']).columns
skewed_features_train_a = X_train_a[numeric_features_train_a].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewed_features_train_a = skewed_features_train_a[abs(skewed_features_train_a) > 0.5]
print(f'Skewed features in training set:\n{skewed_features_train_a}')

for feature_a in skewed_features_train_a.index:
    X_train_a[feature_a] = np.log1p(X_train_a[feature_a])
    X_test_b[feature_a] = np.log1p(X_test_b[feature_a])


Skewed features in training set:
behavioral_antiviral_meds      4.186267
behavioral_face_mask           3.401553
child_under_6_months           3.032832
health_worker                  2.461945
household_children             1.543815
doctor_recc_xyz                1.349659
chronic_med_condition          0.962041
opinion_seas_sick_from_vacc    0.919535
household_adults               0.786528
doctor_recc_seasonal           0.724353
behavioral_outside_home        0.688189
opinion_xyz_risk               0.672918
opinion_xyz_sick_from_vacc     0.651243
behavioral_large_gatherings    0.589489
behavioral_touch_face         -0.758312
opinion_xyz_vacc_effective    -0.902654
behavioral_avoidance          -1.011249
opinion_seas_vacc_effective   -1.315101
behavioral_wash_hands         -1.716283
dtype: float64


In [5]:
numeric_features_train_a = X_train_a.select_dtypes(include=['int64', 'float64']).columns
categorical_features_train_a = X_train_a.select_dtypes(include=['object', 'category']).columns

numeric_transformer_a = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer_a = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor_train_a = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_a, numeric_features_train_a),
        ('cat', categorical_transformer_a, categorical_features_train_a)
    ])


In [6]:
X_train_preprocessed_a = preprocessor_train_a.fit_transform(X_train_a)
X_test_preprocessed_b = preprocessor_train_a.transform(X_test_b)


In [7]:
rf_model_h1n1_a = RandomForestClassifier(random_state=42)
rf_model_seasonal_a = RandomForestClassifier(random_state=42)

cv_a = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rf_model_h1n1_a.fit(X_train_preprocessed_a, y_train_a['xyz_vaccine'])
h1n1_scores_a = cross_val_score(rf_model_h1n1_a, X_train_preprocessed_a, y_train_a['xyz_vaccine'], cv=cv_a, scoring='roc_auc')
mean_h1n1_auc_a = np.mean(h1n1_scores_a)

rf_model_seasonal_a.fit(X_train_preprocessed_a, y_train_a['seasonal_vaccine'])
seasonal_scores_a = cross_val_score(rf_model_seasonal_a, X_train_preprocessed_a, y_train_a['seasonal_vaccine'], cv=cv_a, scoring='roc_auc')
mean_seasonal_auc_a = np.mean(seasonal_scores_a)

mean_auc_a = np.mean([mean_h1n1_auc_a, mean_seasonal_auc_a])

print(f'Mean ROC AUC for h1n1_vaccine: {mean_h1n1_auc_a}')
print(f'Mean ROC AUC for seasonal_vaccine: {mean_seasonal_auc_a}')
print(f'Mean ROC AUC for both vaccines: {mean_auc_a}')

y_pred_h1n1_test_a = rf_model_h1n1_a.predict_proba(X_test_preprocessed_b)[:, 1]
y_pred_seasonal_test_a = rf_model_seasonal_a.predict_proba(X_test_preprocessed_b)[:, 1]

submission_a = pd.DataFrame({
    'respondent_id': test_id_b,
    'xyz_vaccine': y_pred_h1n1_test_a,
    'seasonal_vaccine': y_pred_seasonal_test_a
})

submission_a.to_csv('submission.csv', index=False)


Mean ROC AUC for h1n1_vaccine: 0.8223687226584335
Mean ROC AUC for seasonal_vaccine: 0.8467393955910418
Mean ROC AUC for both vaccines: 0.8345540591247377
